# LLM-based event detection

In [1]:
# pip install openai

In [2]:
# pip install fastapi-poe

In [3]:
import os
from dotenv import load_dotenv
import openai
import pandas as pd

from src.vars import OUTPUT_DIR, ROOT_DIR
import json
import re
import time
import openpyxl
import ast
import random
from datetime import datetime

In [4]:
EXECUTE_LLM = False  # to do not waste resources by mistake
EXECUTE_PRELIMINARY_PHASE = False
EXTRACT_PRELIMINARY_SAMPLE = False
EXTRACT_EVALUATION_SAMPLE = False

In [5]:
df = pd.read_csv(f'{OUTPUT_DIR}/df_complementary_info_filter_dates_keywords_acronims.csv')

In [6]:
df_llm_output = pd.DataFrame(columns=['case_id', 'output'])
if os.path.exists(f'{OUTPUT_DIR}/df_llm_output.csv'):
    df_llm_output = pd.read_csv(f'{OUTPUT_DIR}/df_llm_output.csv')

if EXECUTE_PRELIMINARY_PHASE:
    df_llm_output = pd.DataFrame(columns=['case_id', 'output'])

df_llm_output

,case_id,output
0,219-630869,"['[{""event"": ""DDG n. 36"", ""date"": ""31/05/2022""..."
1,123-350412,['[]']
2,153-437453,"['[{""event"": ""DGC n. 567"", ""date"": ""07/07/2022..."
3,243-697432,"['[{""event"": ""DD n. 73/2022"", ""date"": ""28/10/2..."
4,035-089020,['[]']
...,...,...
595,118-333509,['[]']
596,102-284909,['[]']
597,114-321551,['[]']
598,248-722796,['[]']


In [7]:
def extract_json_objects(text):
    # Pattern to find potential JSON objects (objects and arrays)
    array_pattern = r'(\[[^\[\]]*(\[[^\[\]]*\][^\[\]]*)*\])'

    # Find all potential matches
    assert type(text) == str
    potential_arrays = re.findall(array_pattern, text)

    # Extract just the matched strings
    potential_jsons = [match[0] for match in potential_arrays]

    # Try to parse each potential match
    valid_jsons = []

    for json_str in potential_jsons:
        try:
            # Check if it's valid JSON
            json_obj = json.loads(json_str)
            valid_jsons.append(json_str)
        except json.JSONDecodeError:
            # Skip invalid JSON
            continue

    return [json.dumps(json.loads(obj)) for obj in valid_jsons]

In [8]:
sys_prompt = ''
with open(f'{ROOT_DIR}/res/prompts/System Prompt.txt', encoding='utf-8') as file:
    sys_prompt = file.read()

In [9]:
if EXTRACT_PRELIMINARY_SAMPLE:
    idx_sample = df.sample(n=10).case_id
    idx_sample.to_csv(f'{OUTPUT_DIR}/preliminary_evaluation/idx_sample.csv', index=False)
    df = df[df.case_id.isin(idx_sample)]
elif EXECUTE_PRELIMINARY_PHASE:
    idx_sample = pd.read_csv(f'{OUTPUT_DIR}/preliminary_evaluation/idx_sample.csv').case_id
    df = df[df.case_id.isin(idx_sample)]
else:
    df = df.head(600)

In [10]:
df

,case_id,complementary_info
0,219-630869,Atto esito di gara: Provvedimento DG n. 36 del...
1,123-350412,RUP: dott.ssa Maria Lomboni. Delibera di aggiu...
2,153-437453,"1) il bando di gara, il fac-simile modulo di o..."
3,243-697432,Determinazione Agg.ne AD di Marche Multiserviz...
4,035-089020,1) La procedura è gestita con il Sistema telem...
...,...,...
595,118-333509,Determinazioni a contrarre e di approvazione d...
596,102-284909,La ricezione delle richieste di partecipazione...
597,114-321551,La procedura di scelta del contraente si svolg...
598,248-722796,La presente procedura viene gestita interament...


In [11]:
def run_prompt(user_message: str, model: str):
    time.sleep(3)
    result = dict()

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": user_message}],
        stream=True
    )
    result[model] = ''
    for chunk in response:
        if chunk.choices[0].delta.content:
            result[model] += chunk.choices[0].delta.content
    return result

def analyse_instances(df, model, idx):
    executed_caseid = df_llm_output['case_id'].tolist()

    for index, row in df.iterrows():
        if row.case_id in executed_caseid:
            continue
        print(f"Row id: {index}")

        prompt = prompts[idx]
        user_message = prompt % (df[df['case_id'] == row.case_id].iloc[0]['complementary_info'])
        new_row = dict()
        result = run_prompt(user_message, model)

        json_objs = extract_json_objects(result[model])
        if len(json_objs) == 1:
            json_objects = json_objs[0]
        elif len(json_objs) == 0:
            result = run_prompt(user_message, model)
            json_objs = extract_json_objects(result[model])
            if len(json_objs) == 1:
                json_objects = json_objs[0]
            elif len(json_objs) == 0:
                print(f"Error: no json objects found again in instance {index} -> '{row}'")
                continue

        new_row = {"case_id": row.case_id, "output": json_objs}
        df_llm_output.loc[len(df_llm_output)] = new_row

        if index % 5 == 0:
            print(f"TED number {index} / {len(df.index)}...")
            if EXECUTE_PRELIMINARY_PHASE:
                df_llm_output.to_csv(f'{OUTPUT_DIR}/preliminary_evaluation/prompt_{idx}_{model}.csv', index=False)
            else:
                df_llm_output.to_csv(f'{OUTPUT_DIR}/df_llm_output.csv', index=False)

## Prompt validation, model selection, execution

In [12]:
prompts = ["", "", ""]
with open(f'{ROOT_DIR}/res/prompts/prompt1 - zero shot.txt', encoding='utf-8') as file:
    prompts[0] = file.read()

In [13]:
prompts[1] = ""
with open(f'{ROOT_DIR}/res/prompts/prompt2 - few shot.txt', encoding='utf-8') as file:
    prompts[1] = file.read()

In [14]:
prompts[2] = ""
with open(f'{ROOT_DIR}/res/prompts/prompt3 - chain of thoughts.txt', encoding='utf-8') as file:
    prompts[2] = file.read()

In [15]:
if EXECUTE_LLM:
    load_dotenv()
    client = openai.OpenAI(
      api_key=os.getenv('POE_API_KEY'),
      base_url="https://api.poe.com/v1",
    )

    if EXECUTE_PRELIMINARY_PHASE:
        models = ["Llama-3.1-8B", "GPT-4o", "Claude-Sonnet-4"]

        for idx, prompt in enumerate(prompts):
            for model in models:
                print(f"{model} - prompt {idx}")
                analyse_instances(df, model, idx)
    else:
        model = "Claude-Sonnet-4"
        idx = 2
        print(f"{model} - prompt {idx}")
        analyse_instances(df, model, idx)

In [16]:
df_llm_output

,case_id,output
0,219-630869,"['[{""event"": ""DDG n. 36"", ""date"": ""31/05/2022""..."
1,123-350412,['[]']
2,153-437453,"['[{""event"": ""DGC n. 567"", ""date"": ""07/07/2022..."
3,243-697432,"['[{""event"": ""DD n. 73/2022"", ""date"": ""28/10/2..."
4,035-089020,['[]']
...,...,...
595,118-333509,['[]']
596,102-284909,['[]']
597,114-321551,['[]']
598,248-722796,['[]']


In [17]:
print("Correct instances (manual annotation) after the Preliminary Phase:")

print("\nPrompt 0 - Claude-Sonnet-4: 8/10")
print("Prompt 0 - GPT-4o: 7/10")
print("Prompt 0 - Llama-3.1-8B: 2/10")

print("\nPrompt 1 - Claude-Sonnet-4: 7/10")
print("Prompt 1 - GPT-4o: 7/10")
print("Prompt 1 - Llama-3.1-8B: 6/10")

print("\nPrompt 2 - Claude-Sonnet-4: 9/10")
print("Prompt 2 - GPT-4o: 8/10")
print("Prompt 2 - Llama-3.1-8B: 4/10")

print("Chosen combination: Prompt 2 - Claude-Sonnet-4")

Correct instances (manual annotation) after the Preliminary Phase:

Prompt 0 - Claude-Sonnet-4: 8/10
Prompt 0 - GPT-4o: 7/10
Prompt 0 - Llama-3.1-8B: 2/10

Prompt 1 - Claude-Sonnet-4: 7/10
Prompt 1 - GPT-4o: 7/10
Prompt 1 - Llama-3.1-8B: 6/10

Prompt 2 - Claude-Sonnet-4: 9/10
Prompt 2 - GPT-4o: 8/10
Prompt 2 - Llama-3.1-8B: 4/10
Chosen combination: Prompt 2 - Claude-Sonnet-4


## Evaluation

In [18]:
df_evaluation_temp = df.copy()

In [19]:
def output_handling(data):
    data = ast.literal_eval(data)
    new_data = []
    for item in data:
        new_data.append(ast.literal_eval(item))
    return new_data

def flatten(xss):
    return [x for xs in xss for x in xs]

def remove_duplicate_dicts(list_of_dicts):
    unique_dicts = []
    seen = set()

    for d in list_of_dicts:
        items = frozenset(d.items())
        if items not in seen:
            seen.add(items)
            unique_dicts.append(d)

    return unique_dicts

def remove_old_dates(list_of_dicts):
    start_of_2016 = datetime(2016, 1, 1)
    filtered_list = []

    for d in list_of_dicts:
        if "date" in d:
            try:
                date_obj = datetime.strptime(d["date"], "%d/%m/%Y")
                if date_obj >= start_of_2016:
                    filtered_list.append(d)
            except ValueError:
                print(f"Warning: Invalid date format in: {d}")
        else:
            print(f"Warning: No 'date' key in: {d}")
    return filtered_list

def is_date_before_2016(italian_date_str):
    try:
        date_obj = datetime.strptime(italian_date_str, "%d/%m/%Y")
        start_of_2016 = datetime(2016, 1, 1)
        return date_obj < start_of_2016

    except ValueError:
        print(f"Error: '{italian_date_str}' is not in the correct format. Use DD/MM/YYYY.")
        return None

In [20]:
df_llm_output["output"] = df_llm_output["output"].map(output_handling)
df_llm_output["output"] = df_llm_output["output"].map(flatten)
df_llm_output["output"] = df_llm_output["output"].map(remove_duplicate_dicts)
df_llm_output["output"] = df_llm_output["output"].map(remove_old_dates)
df_llm_output

,case_id,output
0,219-630869,"[{'event': 'DDG n. 36', 'date': '31/05/2022'}]"
1,123-350412,[]
2,153-437453,"[{'event': 'DGC n. 567', 'date': '07/07/2022'}]"
3,243-697432,"[{'event': 'DD n. 73/2022', 'date': '28/10/202..."
4,035-089020,[]
...,...,...
595,118-333509,[]
596,102-284909,[]
597,114-321551,[]
598,248-722796,[]


In [21]:
df_evaluation_temp["event"] = ""
df_evaluation_temp["date"] = ""
df_evaluation_temp["label"] = ""
df_evaluation_temp["note"] = ""
df_evaluation_temp

,case_id,complementary_info,event,date,label,note
0,219-630869,Atto esito di gara: Provvedimento DG n. 36 del...,,,,
1,123-350412,RUP: dott.ssa Maria Lomboni. Delibera di aggiu...,,,,
2,153-437453,"1) il bando di gara, il fac-simile modulo di o...",,,,
3,243-697432,Determinazione Agg.ne AD di Marche Multiserviz...,,,,
4,035-089020,1) La procedura è gestita con il Sistema telem...,,,,
...,...,...,...,...,...,...
595,118-333509,Determinazioni a contrarre e di approvazione d...,,,,
596,102-284909,La ricezione delle richieste di partecipazione...,,,,
597,114-321551,La procedura di scelta del contraente si svolg...,,,,
598,248-722796,La presente procedura viene gestita interament...,,,,


In [22]:
evaluation = list()
for index, row in df_evaluation_temp.iterrows():
    output = df_llm_output.iloc[index]['output']
    if output == []:
        continue
    if row.case_id == "005-009101":
        print()
    for pair in output:
        if is_date_before_2016(pair["event"]):
            continue
        item = dict()
        item["case_id"] = row.case_id
        item["complementary_info"] = row.complementary_info
        item["event"] = pair["event"]
        item["date"] = pair["date"]
        item["label"] = ""
        item["note"] = ""
        evaluation.append(item)

df_evaluation = pd.DataFrame(evaluation)
#df_evaluation = df_evaluation[~(df_evaluation['date'] > '2016-01-01')]
df_evaluation

Error: 'DDG n. 36' is not in the correct format. Use DD/MM/YYYY.
Error: 'DGC n. 567' is not in the correct format. Use DD/MM/YYYY.
Error: 'DD n. 73/2022' is not in the correct format. Use DD/MM/YYYY.
Error: 'Delibera n. 27083' is not in the correct format. Use DD/MM/YYYY.
Error: 'Delibera n. 10' is not in the correct format. Use DD/MM/YYYY.
Error: 'Atto del RUP' is not in the correct format. Use DD/MM/YYYY.
Error: 'DD n. A/D/3504' is not in the correct format. Use DD/MM/YYYY.
Error: 'DD n. 40/2022' is not in the correct format. Use DD/MM/YYYY.
Error: 'DD n. 437' is not in the correct format. Use DD/MM/YYYY.
Error: 'DD n. 92' is not in the correct format. Use DD/MM/YYYY.
Error: 'DDG n. 1311' is not in the correct format. Use DD/MM/YYYY.
Error: 'DM infrastrutture e trasporti' is not in the correct format. Use DD/MM/YYYY.
Error: 'Disposizione a Contrarre' is not in the correct format. Use DD/MM/YYYY.
Error: 'DCC n. 339' is not in the correct format. Use DD/MM/YYYY.
Error: 'DD n. 478' is n

,case_id,complementary_info,event,date,label,note
0,219-630869,Atto esito di gara: Provvedimento DG n. 36 del...,DDG n. 36,31/05/2022,,
1,153-437453,"1) il bando di gara, il fac-simile modulo di o...",DGC n. 567,07/07/2022,,
2,243-697432,Determinazione Agg.ne AD di Marche Multiserviz...,DD n. 73/2022,28/10/2022,,
3,091-251847,a) deliberazione a contrattare prot. n. 27083 ...,Delibera n. 27083,28/04/2022,,
4,045-117742,Deliberazione di indizione della procedura ape...,Delibera n. 10,23/02/2022,,
...,...,...,...,...,...,...
297,162-460604,"CIG 93635451C7\nRUP: avv. Francesca Leone, dip...",Delibera n. 22AU032,09/08/2022,,
298,061-161865,IL VALORE STIMATO E' AL NETTO DI IVA E DEGLI O...,DGC n. 946,21/03/2022,,
299,025-060373,CIG 9046840889\nTermine per ottenere chiarimen...,Decreto n. 841,31/01/2022,,
300,135-381854,"Il Disciplinare di Gara, che forma parte integ...",Decreto n. 236,08/07/2022,,


In [23]:
if EXTRACT_EVALUATION_SAMPLE:
    idx_sample = df_evaluation.case_id.tolist()
    idx_sample = list(set(idx_sample))
    idx_sample = random.sample(idx_sample, 100)
    idx_sample = pd.DataFrame(idx_sample, columns=["case_id"])
    idx_sample.to_csv(f'{OUTPUT_DIR}/evaluation/idx_sample.csv', index=False)

idx_sample = pd.read_csv(f'{OUTPUT_DIR}/evaluation/idx_sample.csv').case_id
df_evaluation = df_evaluation[df_evaluation.case_id.isin(idx_sample)]
df_evaluation

,case_id,complementary_info,event,date,label,note
1,153-437453,"1) il bando di gara, il fac-simile modulo di o...",DGC n. 567,07/07/2022,,
2,243-697432,Determinazione Agg.ne AD di Marche Multiserviz...,DD n. 73/2022,28/10/2022,,
4,045-117742,Deliberazione di indizione della procedura ape...,Delibera n. 10,23/02/2022,,
6,135-384701,Determinazione di contrarre n. A/D/3504 del 07...,DD n. A/D/3504,07/07/2022,,
12,221-636448,La “Disposizione a Contrarre” per la presente ...,Disposizione a Contrarre,02/09/2022,,
...,...,...,...,...,...,...
288,020-050569,Ulteriori informazioni sulla procedura di aggi...,DD n. 246,24/11/2021,,
292,148-423793,a) A pena di irregolarità non sanabile delle o...,DD Prot. n. CDG-0513635-Int.,22/07/2022,,
297,162-460604,"CIG 93635451C7\nRUP: avv. Francesca Leone, dip...",Delibera n. 22AU032,09/08/2022,,
298,061-161865,IL VALORE STIMATO E' AL NETTO DI IVA E DEGLI O...,DGC n. 946,21/03/2022,,


In [24]:
with pd.ExcelWriter(f'{OUTPUT_DIR}/evaluation/TED_annotation.xlsx') as writer:
        df_evaluation.to_excel(writer, sheet_name='event_detection')